In [1]:
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


In [2]:
import pandas as pd
import sqlite3

In [3]:
game_log = pd.read_csv('game_log.csv')
park_codes = pd.read_csv('park_codes.csv')
person_codes = pd.read_csv('person_codes.csv')
team_codes = pd.read_csv('team_codes.csv')

/home/isaac/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (12,13,14,15,19,20,81,82,83,84,85,86,87,88,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

In [5]:
game_log.shape

(171907, 161)

In [6]:
game_log.tail()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
171902,20161002,0,Sun,MIL,NL,162,COL,NL,162,6,4,60.0,D,NaN,NaN,NaN,DEN02,27762.0,203.0,0200000202,1100100010,39.0,10.0,4.0,1.0,2.0,6.0,0.0,0.0,1.0,4.0,0.0,12.0,2.0,1.0,0.0,0.0,8.0,7.0,4.0,4.0,1.0,0.0,30.0,12.0,1.0,0.0,0.0,0.0,41.0,13.0,4.0,0.0,1.0,4.0,1.0,0.0,1.0,3.0,0.0,11.0,0.0,1.0,0.0,0.0,12.0,5.0,6.0,6.0,0.0,0.0,30.0,13.0,0.0,0.0,0.0,0.0,barrs901,Scott Barry,woodt901,Tom Woodring,randt901,Tony Randazzo,ortir901,Roberto Ortiz,NaN,NaN,NaN,NaN,counc001,Craig Counsell,weisw001,Walt Weiss,thort001,Tyler Thornburg,rusic001,Chris Rusin,knebc001,Corey Knebel,susaa001,Andrew Susac,cravt001,Tyler Cravy,marqg001,German Marquez,villj001,Jonathan Villar,5.0,genns001,Scooter Gennett,4.0,cartc002,Chris Carter,3.0,santd002,Domingo Santana,9.0,pereh001,Hernan Perez,8.0,arcio002,Orlando Arcia,6.0,susaa001,Andrew Susac,2.0,elmoj001,Jake Elmore,7.0,cravt001,Tyler Cravy,1.0,blacc001,Charlie Blackmon,8.0,dahld001,David Dahl,7.0,arenn001,Nolan Arenado,5.0,gonzc001,Carlos Gonzalez,9.0,murpt002,Tom Murphy,2.0,pattj005,Jordan Patterson,3.0,valap001,Pat Valaika,4.0,adamc001,Cristhian Adames,6.0,marqg001,German Marquez,1.0,NaN,Y
171903,20161002,0,Sun,NYN,NL,162,PHI,NL,162,2,5,51.0,D,NaN,NaN,NaN,PHI13,36935.0,159.0,000001100,00100031x,33.0,8.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,9.0,1.0,1.0,1.0,0.0,6.0,6.0,3.0,3.0,0.0,0.0,24.0,12.0,3.0,1.0,2.0,0.0,33.0,10.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,2.0,0.0,3.0,0.0,0.0,2.0,0.0,7.0,5.0,2.0,2.0,0.0,0.0,27.0,7.0,0.0,0.0,1.0,0.0,barkl901,Lance Barksdale,herna901,Angel Hernandez,barrt901,Ted Barrett,littw901,Will Little,NaN,NaN,NaN,NaN,collt801,Terr

In [7]:
park_codes.shape

(252, 9)

In [8]:
person_codes.shape

(20494, 7)

In [9]:
park_codes.head()

,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,NaN,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,NaN,Altoona,PA,04/30/1884,05/31/1884,UA,NaN
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,04/19/1966,NaN,AL,NaN
3,ARL01,Arlington Stadium,NaN,Arlington,TX,04/21/1972,10/03/1993,AL,NaN
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,04/11/1994,NaN,AL,NaN


In [10]:
team_codes.shape

(150, 8)

In [11]:
team_codes

,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1
1,ARI,NL,1998,0,Arizona,Diamondbacks,ARI,1
2,BFN,NL,1879,1885,Buffalo,Bisons,BFN,1
3,BFP,PL,1890,1890,Buffalo,Bisons,BFP,1
4,BL1,NaN,1872,1874,Baltimore,Canaries,BL1,1
5,BL2,AA,1882,1891,Baltimore,Orioles,BL2,1
6,BLN,NL,1892,1899,Baltimore,Orioles,BL2,2
7,BL4,NaN,1873,1873,Baltimore,Marylands,BL4,1
8,BLA,AL,1901,1902,Baltimore,Orioles,BLA,1
9,NYA,AL,1903,0,New York,Yankees,BLA,2


In [12]:
person_codes.head()

,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,04/06/2004,NaN,NaN,NaN
1,aaroh101,Aaron,Hank,04/13/1954,NaN,NaN,NaN
2,aarot101,Aaron,Tommie,04/10/1962,NaN,04/06/1979,NaN
3,aased001,Aase,Don,07/26/1977,NaN,NaN,NaN
4,abada001,Abad,Andy,09/10/2001,NaN,NaN,NaN


### Creating the Database:

In [13]:
def run_query(q):
    with sqlite3.connect('mlb.db') as conn:
        return pd.read_sql(q, conn)
def run_command(command):
    with sqlite3.connect('mlb.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(command)

In [14]:
conn = sqlite3.connect("mlb.db")
# game_log.to_sql('game_log', conn, index=False)

In [15]:
# park_codes.to_sql('park_codes', conn, index=False)
# person_codes.to_sql('person_codes', conn, index=False)
# team_codes.to_sql('team_codes', conn, index=False)

In [16]:
# run_command("ALTER TABLE game_log ADD COLUMN game_id;")

In [17]:
# run_command("UPDATE game_log SET game_id = h_name||date||h_game_number")

In [18]:
game_log.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,2,54.0,D,NaN,NaN,NaN,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,6.0,1.0,NaN,-1.0,NaN,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,NaN,-1.0,NaN,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,NaN,NaN,NaN,NaN,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,NaN,1,WS3,NaN,1,20,18,54.0,D,NaN,NaN,NaN,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,NaN,5.0,3.0,NaN,-1.0,NaN,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,NaN,2.0,1.0,NaN,-1.0,NaN,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,NaN,NaN,NaN,NaN,spala101,Al Spalding,braia102,Asa Brainard,wrigg101,George Wright,6.0,barnr102,Ross

In [19]:
run_query("SELECT h_name, date, h_game_number, game_id FROM game_log LIMIT 5")

,h_name,date,h_game_number,game_id
0,FW1,18710504,1,FW1187105041
1,WS3,18710505,1,WS3187105051
2,RC1,18710506,1,RC1187105061
3,CH1,18710508,1,CH1187105081
4,TRO,18710509,1,TRO187105091


In [20]:
game_log.columns

Index(['date', 'number_of_game', 'day_of_week', 'v_name', 'v_league',
       'v_game_number', 'h_name', 'h_league', 'h_game_number', 'v_score',
       ...
       'h_player_7_name', 'h_player_7_def_pos', 'h_player_8_id',
       'h_player_8_name', 'h_player_8_def_pos', 'h_player_9_id',
       'h_player_9_name', 'h_player_9_def_pos', 'additional_info',
       'acquisition_info'],
      dtype='object', length=161)

In [21]:
person_create = """
CREATE TABLE IF NOT EXISTS person (
    person_id TEXT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT
);
"""
run_command(person_create)

In [22]:
person_add = """
INSERT OR IGNORE INTO person
    SELECT id, first, last FROM person_codes
"""
run_command(person_add)

In [23]:
run_query("SELECT * FROM person LIMIT 10")

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad
5,abadf001,Fernando,Abad
6,abadj101,John,Abadie
7,abbae101,Ed,Abbaticchio
8,abbeb101,Bert,Abbey
9,abbec101,Charlie,Abbey


In [24]:
park_create = """
CREATE TABLE IF NOT EXISTS park (
    park_id TEXT PRIMARY KEY,
    name TEXT,
    nickname TEXT,
    city TEXT,
    state TEXT,
    notes TEXT
)
"""
run_command(park_create)

In [25]:
run_command("""
INSERT OR IGNORE INTO park
    SELECT park_id, name, aka, city, state, notes FROM park_codes
""")

In [26]:
run_query("SELECT * FROM park LIMIT 10")

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None
5,ATL01,Atlanta-Fulton County Stadium,None,Atlanta,GA,None
6,ATL02,Turner Field,None,Atlanta,GA,None
7,ATL03,Suntrust Park,None,Atlanta,GA,None
8,BAL01,Madison Avenue Grounds,None,Baltimore,MD,WS3
9,BAL02,Newington Park,None,Baltimore,MD,BL1:1872-74; BL4:1873; BL2: 1882


In [27]:
run_query("SELECT * FROM team_codes WHERE league = 'FL'")

,team_id,league,start,end,city,nickname,franch_id,seq
0,BLF,FL,1914,1915,Baltimore,Terrapins,BLF,1
1,BRF,FL,1914,1915,Brooklyn,Tip-Tops,BRF,1
2,BUF,FL,1914,1915,Buffalo,Feds,BUF,1
3,CHF,FL,1914,1915,Chicago,Whales,CHF,1
4,IND,FL,1914,1914,Indianapolis,Hoosier-Feds,IND,1
5,NEW,FL,1915,1915,Newark,Peppers,IND,2
6,KCF,FL,1914,1915,Kansas City,Packers,KCF,1
7,PTF,FL,1914,1915,Pittsburgh,Rebels,PTF,1
8,SLF,FL,1914,1915,St. Louis,Terriers,SLF,1


In [28]:
run_command("""
CREATE TABLE IF NOT EXISTS league(
    league_id TEXT PRIMARY KEY,
    name TEXT
);
""")

In [29]:
run_command("""
INSERT OR IGNORE INTO league
    VALUES
        ('UA', 'Union Association'),
        ('NL', 'National League'),
        ('PL', 'Players League'),
        ('AA', 'Double-A'),
        ('AL', 'American League'),
        ('FL', 'Federal League');
""")

In [30]:
run_query("SELECT * FROM league")

,league_id,name
0,UA,Union Association
1,NL,National League
2,PL,Players League
3,AA,Double-A
4,AL,American League
5,FL,Federal League


In [31]:
appearance_df = pd.read_csv('appearance_type.csv')
appearance_df

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense
5,O6,Batter 6,offense
6,O7,Batter 7,offense
7,O8,Batter 8,offense
8,O9,Batter 9,offense
9,D1,Pitcher,defense


In [32]:
run_command("""
CREATE TABLE IF NOT EXISTS appearance_type(
    appearance_type TEXT PRIMARY KEY,
    name TEXT,
    category TEXT
)
""")

In [33]:
# appearance_df.to_sql('appear_inter', conn, index=False)

In [34]:
run_command("""
INSERT OR IGNORE INTO appearance_type
    SELECT * FROM appear_inter
""")

In [55]:
run_query('SELECT * FROM appearance_type').iloc[19:]

,appearance_type,name,category
19,UHP,Home Plate,umpire
20,U1B,First Base,umpire
21,U2B,Second Base,umpire
22,U3B,Third Base,umpire
23,ULF,Left Field,umpire
24,URF,Right Field,umpire
25,MM,Manager,manager
26,AWP,Winning Pitcher,award
27,ALP,Losing Pitcher,award
28,ASP,Saving Pitcher,award


In [36]:
run_command("""
CREATE TABLE IF NOT EXISTS team(
    team_id TEXT PRIMARY KEY,
    league_id TEXT,
    city TEXT,
    nickname TEXT,
    franch_id TEXT,
    FOREIGN KEY (league_id) REFERENCES league(league_id)
)
""")

In [37]:
team_codes.head()

,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1
1,ARI,NL,1998,0,Arizona,Diamondbacks,ARI,1
2,BFN,NL,1879,1885,Buffalo,Bisons,BFN,1
3,BFP,PL,1890,1890,Buffalo,Bisons,BFP,1
4,BL1,NaN,1872,1874,Baltimore,Canaries,BL1,1


In [38]:
run_command("""
INSERT OR IGNORE INTO team
    SELECT team_id, league, city, nickname, franch_id FROM team_codes
""")

In [39]:
run_query("SELECT * FROM team LIMIT 10")

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1
5,BL2,AA,Baltimore,Orioles,BL2
6,BLN,NL,Baltimore,Orioles,BL2
7,BL4,None,Baltimore,Marylands,BL4
8,BLA,AL,Baltimore,Orioles,BLA
9,NYA,AL,New York,Yankees,BLA


In [40]:
run_query("SELECT * FROM game_log LIMIT 15")

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info,game_id
0,18710504,0,Thu,CL1,None,1,FW1,None,1,0,2,54.0,D,None,None,None,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,None,6.0,1.0,None,-1.0,None,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,None,0.0,0.0,None,-1.0,None,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,None,None,None,None,None,None,None,None,None,None,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,None,None,None,None,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,None,Y,FW1187105041
1,18710505,0,Fri,BS1,None,1,WS3,None,1,20,18,54.0,D,None,None,None,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,None,5.0,3.0,None,-1.0,None,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,None,2.0,1.0,None,-1.0,None,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,None,None,None,None,None,None,None,None,None,None,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,None,None,None,None,spala101,Al

In [41]:
run_command("""
CREATE TABLE IF NOT EXISTS game(
    game_id TEXT PRIMARY KEY,
    date INTEGER,
    number_of_game INTEGER,
    park_id TEXT,
    length_outs REAL,
    day BLOB,
    completion TEXT,
    forfeit TEXT,
    protest TEXT,
    attendance REAL,
    length_minutes REAL,
    additional_info TEXT,
    acquisition_info TEXT,
    FOREIGN KEY (park_id) REFERENCES park(park_id)
)
""")

In [42]:
run_command("""
INSERT OR IGNORE INTO game
    SELECT game_id, date, number_of_game, park_id, length_outs,
        CASE
            WHEN day_night = 'D' THEN 1
            WHEN day_night = 'N' THEN 0
            
        END,
        completion,
        forefeit,
        protest,
        attendance,
        length_minutes,
        additional_info,
        acquisition_info
    FROM game_log
""")

In [43]:
run_query("SELECT * FROM game LIMIT 10")

,game_id,date,number_of_game,park_id,length_outs,day,completion,forfeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105041,18710504,0,FOR01,54.0,1,None,None,None,200.0,120.0,None,Y
1,WS3187105051,18710505,0,WAS01,54.0,1,None,None,None,5000.0,145.0,HTBF,Y
2,RC1187105061,18710506,0,RCK01,54.0,1,None,None,None,1000.0,140.0,None,Y
3,CH1187105081,18710508,0,CHI01,54.0,1,None,None,None,5000.0,150.0,None,Y
4,TRO187105091,18710509,0,TRO01,54.0,1,None,None,None,3250.0,145.0,HTBF,Y
5,CL1187105114,18710511,0,CLE01,48.0,1,None,V,None,2500.0,120.0,None,Y
6,CL1187105135,18710513,0,CIN01,54.0,1,None,None,None,1200.0,150.0,None,Y
7,FW1187105132,18710513,0,FOR01,54.0,1,None,None,None,1500.0,105.0,None,Y
8,FW1187105153,18710515,0,FOR01,54.0,1,None,None,None,NaN,140.0,None,Y
9,BS1187105163,18710516,0,BOS01,54.0,1,None,None,None,2500.0,NaN,HTBF,Y


In [44]:
column_list = run_query("""
SELECT sql
FROM (SELECT sql FROM sqlite_master WHERE name = 'game_log' AND type = 'table')

WHERE sql LIKE '%player%'
""")['sql'].iloc[0].replace('CREATE TABLE "game_log" (\n','').replace('\n, game_id)','').replace('"','').split(',\n  ')

In [45]:
[h[2:] for h in column_list if ('h_' in h) and ('length_' not in h)]

['name TEXT',
 'league TEXT',
 'game_number INTEGER',
 'score INTEGER',
 'line_score TEXT',
 'at_bats REAL',
 'hits REAL',
 'doubles REAL',
 'triples REAL',
 'homeruns REAL',
 'rbi REAL',
 'sacrifice_hits REAL',
 'sacrifice_flies REAL',
 'hit_by_pitch REAL',
 'walks REAL',
 'intentional_walks REAL',
 'strikeouts REAL',
 'stolen_bases REAL',
 'caught_stealing REAL',
 'grounded_into_double REAL',
 'first_catcher_interference REAL',
 'left_on_base REAL',
 'pitchers_used REAL',
 'individual_earned_runs REAL',
 'team_earned_runs REAL',
 'wild_pitches REAL',
 'balks REAL',
 'putouts REAL',
 'assists REAL',
 'errors REAL',
 'passed_balls REAL',
 'double_plays REAL',
 'triple_plays REAL',
 'manager_id TEXT',
 'manager_name TEXT',
 'starting_pitcher_id TEXT',
 'starting_pitcher_name TEXT',
 'player_1_id TEXT',
 'player_1_name TEXT',
 'player_1_def_pos REAL',
 'player_2_id TEXT',
 'player_2_name TEXT',
 'player_2_def_pos REAL',
 'player_3_id TEXT',
 'player_3_name TEXT',
 'player_3_def_pos REAL'

In [46]:
strr = """'score INTEGER', 'line_score TEXT', 'at_bats REAL', 'hits REAL', 'doubles REAL', 'triples REAL', 'homeruns REAL', 'rbi REAL', 'sacrifice_hits REAL', 'sacrifice_flies REAL', 'hit_by_pitch REAL', 'walks REAL', 'intentional_walks REAL', 'strikeouts REAL', 'stolen_bases REAL', 'caught_stealing REAL', 'grounded_into_double REAL', 'first_catcher_interference REAL', 'left_on_base REAL', 'pitchers_used REAL', 'individual_earned_runs REAL', 'team_earned_runs REAL', 'wild_pitches REAL', 'balks REAL', 'putouts REAL', 'assists REAL', 'errors REAL', 'passed_balls REAL', 'double_plays REAL', 'triple_plays REAL'"""

In [47]:
strr.replace('\'','')

'score INTEGER, line_score TEXT, at_bats REAL, hits REAL, doubles REAL, triples REAL, homeruns REAL, rbi REAL, sacrifice_hits REAL, sacrifice_flies REAL, hit_by_pitch REAL, walks REAL, intentional_walks REAL, strikeouts REAL, stolen_bases REAL, caught_stealing REAL, grounded_into_double REAL, first_catcher_interference REAL, left_on_base REAL, pitchers_used REAL, individual_earned_runs REAL, team_earned_runs REAL, wild_pitches REAL, balks REAL, putouts REAL, assists REAL, errors REAL, passed_balls REAL, double_plays REAL, triple_plays REAL'

In [48]:
run_command("""
CREATE TABLE IF NOT EXISTS team_appearance(
    team_id TEXT,
    game_id TEXT,
    home BLOB,
    league_id TEXT,
    score INTEGER, line_score TEXT, at_bats REAL, hits REAL, doubles REAL, triples REAL, homeruns REAL, rbi REAL, sacrifice_hits REAL, sacrifice_flies REAL, hit_by_pitch REAL, walks REAL, intentional_walks REAL, strikeouts REAL, stolen_bases REAL, caught_stealing REAL, grounded_into_double REAL, first_catcher_interference REAL, left_on_base REAL, pitchers_used REAL, individual_earned_runs REAL, team_earned_runs REAL, wild_pitches REAL, balks REAL, putouts REAL, assists REAL, errors REAL, passed_balls REAL, double_plays REAL, triple_plays REAL,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (league_id) REFERENCES league(league_id)
);
""")

In [49]:
# run_command("DROP TABLE team_appearance")

In [50]:
", ".join([h for h in column_list if ('h_' in h) and ('length_' not in h)][3:33])
# Used this to combine home columns into string which then I copy and pasted into the INSERT INTO command

'h_score INTEGER, h_line_score TEXT, h_at_bats REAL, h_hits REAL, h_doubles REAL, h_triples REAL, h_homeruns REAL, h_rbi REAL, h_sacrifice_hits REAL, h_sacrifice_flies REAL, h_hit_by_pitch REAL, h_walks REAL, h_intentional_walks REAL, h_strikeouts REAL, h_stolen_bases REAL, h_caught_stealing REAL, h_grounded_into_double REAL, h_first_catcher_interference REAL, h_left_on_base REAL, h_pitchers_used REAL, h_individual_earned_runs REAL, h_team_earned_runs REAL, h_wild_pitches REAL, h_balks REAL, h_putouts REAL, h_assists REAL, h_errors REAL, h_passed_balls REAL, h_double_plays REAL, h_triple_plays REAL'

In [51]:
run_command("""
INSERT OR IGNORE INTO team_appearance
    SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score, h_line_score, h_at_bats, h_hits,
        h_doubles, h_triples, h_homeruns, h_rbi, h_sacrifice_hits, h_sacrifice_flies, h_hit_by_pitch,
        h_walks, h_intentional_walks, h_strikeouts, h_stolen_bases, h_caught_stealing, h_grounded_into_double,
        h_first_catcher_interference, h_left_on_base, h_pitchers_used, h_individual_earned_runs,
        h_team_earned_runs, h_wild_pitches, h_balks, h_putouts, h_assists, h_errors,
        h_passed_balls, h_double_plays, h_triple_plays
    FROM game_log
UNION
    SELECT
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score, v_line_score, v_at_bats, v_hits,
        v_doubles, v_triples, v_homeruns, v_rbi, v_sacrifice_hits, v_sacrifice_flies, v_hit_by_pitch,
        v_walks, v_intentional_walks, v_strikeouts, v_stolen_bases, v_caught_stealing, v_grounded_into_double,
        v_first_catcher_interference, v_left_on_base, v_pitchers_used, v_individual_earned_runs,
        v_team_earned_runs, v_wild_pitches, v_balks, v_putouts, v_assists, v_errors,
        v_passed_balls, v_double_plays, v_triple_plays
    FROM game_log
""")

In [52]:
run_query("SELECT * FROM team_appearance WHERE line_score != 'None' LIMIT 20")

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ANA,ANA200004031,1,AL,2,010000001,35.0,10.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,5.0,0.0,6.0,0.0,1.0,0.0,0.0,11.0,3.0,3.0,3.0,0.0,0.0,27.0,16.0,1.0,0.0,0.0,0.0
1,ANA,ANA200004042,1,AL,3,000003000,36.0,10.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,6.0,0.0,9.0,1.0,0.0,2.0,0.0,12.0,4.0,5.0,5.0,0.0,0.0,27.0,12.0,1.0,0.0,2.0,0.0
2,ANA,ANA200004053,1,AL,12,12610110x,33.0,12.0,4.0,0.0,1.0,12.0,0.0,2.0,1.0,8.0,0.0,4.0,1.0,0.0,0.0,0.0,8.0,3.0,6.0,6.0,0.0,0.0,27.0,13.0,2.0,0.0,2.0,0.0
3,ANA,ANA200004074,1,AL,7,30000310x,32.0,9.0,2.0,0.0,2.0,7.0,1.0,0.0,0.0,5.0,2.0,5.0,3.0,0.0,0.0,0.0,7.0,2.0,3.0,3.0,0.0,0.0,27.0,9.0,0.0,0.0,1.0,0.0
4,ANA,ANA200004085,1,AL,7,20000401x,35.0,13.0,4.0,0.0,1.0,7.0,1.0,0.0,0.0,1.0,0.0,4.0,0.0,1.0,0.0,0.0,6.0,6.0,5.0,5.0,2.0,0.0,27.0,18.0,0.0,0.0,3.0,0.0
5,ANA,ANA200004096,1,AL,2,000000011,33.0,7.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,1.0,0.0,14.0,2.0,0.0,0.0,0.0,6.0,3.0,5.0,5.0,0.0,0.0,27.0,8.0,0.0,0.0,0.0,0.0
6,ANA,ANA200004107,1,AL,6,00200040x,28.0,7.0,2.0,0.0,1.0,6.0,0.0,1.0,0.0,9.0,1.0,1.0,0.0,0.0,1.0,0.0,8.0,1.0,0.0,0.0,2.0,0.0,27.0,17.0,1.0,0.0,2.0,0.0
7,ANA,ANA200004118,1,AL,5,04100000x,29.0,8.0,3.0,0.0,2.0,5.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,1.0,1.0,0.0,3.0,5.0,4.0,4.0,1.0,1.0,27.0,8.0,1.0,0.0,2.0,0.0
8,ANA,ANA200004129,1,AL,2,100010000,33.0,9.0,4.0,0.0,0.0,2.0,0.0,0.0,3.0,2.0,0.0,6.0,1.0,1.0,1.0,0.0,9.0,4.0,6.0,6.0,0.0,0.0,27.0,10.0,0.0,0.0,2.0,0.0
9,ANA,ANA2000042420,1,AL,10,21121300x,35.0,14.0,2.0,0.0,4.0,10.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,2.0,0.0,5.0,3.0,3.0,3.0,1.0,0.0,27.0,13.0,2.0,0.0,2.0,0.0


Above I checked to make sure eveything in the table looked correct.. So far so good

In [64]:
run_command("""
CREATE TABLE IF NOT EXISTS person_appearance(
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id TEXT,
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type (appearance_type)
)
""")

In [72]:
run_query('SELECT * FROM appearance_type')

,appearance_type,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense
5,O6,Batter 6,offense
6,O7,Batter 7,offense
7,O8,Batter 8,offense
8,O9,Batter 9,offense
9,D1,Pitcher,defense


In [70]:
run_command("""
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
)
    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL

UNION
    
    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE [1b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL

UNION
    
    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN v_name
            ELSE h_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION
    
    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL
    
UNION
    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL
    """
           )

In [71]:
run_query("SELECT * FROM person_appearance LIMIT 10")

,appearance_id,person_id,team_id,game_id,appearance_type_id
0,1,maplb901,None,ALT188404308,UHP
1,2,curte801,ALT,ALT188404308,MM
2,3,murpj104,ALT,ALT188404308,PSP
3,4,hodnc101,SLU,ALT188404308,PSP
4,5,sullt101,SLU,ALT188404308,MM
5,6,hoopm101,None,ALT188405029,UHP
6,7,curte801,ALT,ALT188405029,MM
7,8,learj102,ALT,ALT188405029,PSP
8,9,sullt101,SLU,ALT188405029,MM
9,10,taylb103,SLU,ALT188405029,PSP


In [73]:
game_log.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,2,54.0,D,NaN,NaN,NaN,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,6.0,1.0,NaN,-1.0,NaN,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,NaN,-1.0,NaN,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,NaN,NaN,NaN,NaN,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,NaN,1,WS3,NaN,1,20,18,54.0,D,NaN,NaN,NaN,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,NaN,5.0,3.0,NaN,-1.0,NaN,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,NaN,2.0,1.0,NaN,-1.0,NaN,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,NaN,NaN,NaN,NaN,spala101,Al Spalding,braia102,Asa Brainard,wrigg101,George Wright,6.0,barnr102,Ross

Now adding the Players:

In [78]:
template = """
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
"""

# run_command(c1)
# run_command(c2)

for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
#        print(template.format(**query_vars))
#         # run commmand is a helper function which runs
#         # a query against our database.
        run_command(template.format(**query_vars))

In [83]:
run_query("SELECT * FROM person_appearance WHERE appearance_type_id LIKE 'D%' LIMIT 100")

,appearance_id,person_id,team_id,game_id,appearance_type_id
0,1646119,erstd001,ANA,ANA200004031,D7
1,1646121,erstd001,ANA,ANA200004042,D7
2,1646123,erstd001,ANA,ANA200004053,D7
3,1646125,erstd001,ANA,ANA200004074,D7
4,1646127,erstd001,ANA,ANA200004085,D7
5,1646129,erstd001,ANA,ANA200004096,D7
6,1646131,erstd001,ANA,ANA200004107,D7
7,1646133,erstd001,ANA,ANA200004118,D7
8,1646135,erstd001,ANA,ANA200004129,D7
9,1646137,erstd001,ANA,ANA2000042420,D7


In [85]:
run_command("DROP TABLE game_log")

In [86]:
lisst = ['park_','team_','person_']

for el in lisst:
    run_command("DROP TABLE %scodes" % el)

In [87]:
run_query("SELECT * FROM sqlite_master")

,type,name,tbl_name,rootpage,sql
0,table,person,person,37332,"CREATE TABLE person (\n person_id TEXT PRIMARY KEY,\n first_name TEXT,\n last_name TEXT\n)"
1,index,sqlite_autoindex_person_1,person,37333,None
2,table,park,park,37567,"CREATE TABLE park (\n park_id TEXT PRIMARY KEY,\n name TEXT,\n nickname TEXT,\n city TEXT,\n state TEXT,\n notes TEXT\n)"
3,index,sqlite_autoindex_park_1,park,37568,None
4,table,league,league,37573,"CREATE TABLE league(\n league_id TEXT PRIMARY KEY,\n name TEXT\n)"
5,index,sqlite_autoindex_league_1,league,37574,None
6,table,appearance_type,appearance_type,37575,"CREATE TABLE appearance_type(\n appearance_type TEXT PRIMARY KEY,\n name TEXT,\n category TEXT\n)"
7,index,sqlite_autoindex_appearance_type_1,appearance_type,37576,None
8,table,appear_inter,appear_inter,37577,"CREATE TABLE ""appear_inter"" (\n""appearance_type_id"" TEXT,\n ""name"" TEXT,\n ""category"" TEXT\n)"
9,table,team,team,37578,"CREATE TABLE team(\n team_id TEXT PRIMARY KEY,\n league_id TEXT,\n city TEXT,\n nickname TEXT,\n franch_id TEXT,\n FOREIGN KEY (league_id) REFERENCES league(league_id)\n)"
